### Twilight Struggle

In [1]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

### Game Instance

In [2]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()

Initalising game..

----- USSR Player: -----
You may add 6 influence in these countries. Type in their country indices, separated by commas (no spaces).
20	Yugoslavia
14	Austria
18	Czechoslovakia
22	Bulgaria
19	Hungary
21	Romania
17	Poland
7	Finland
16	East_Germany
> q

----- US Player: -----
You may add 7 influence in these countries. Type in their country indices, separated by commas (no spaces).
8	Denmark
12	Italy
3	Canada
15	West_Germany
10	France
14	Austria
6	Sweden
13	Greece
4	UK
23	Turkey
11	Spain_Portugal
9	Benelux
7	Finland
5	Norway
> q

----- US Player: -----
You may add 2 influence in these countries. Type in their country indices, separated by commas (no spaces).
3	Canada
4	UK
26	Israel
30	Iran
42	Australia
44	Philippines
45	Japan
47	South_Korea
59	South_Africa
72	Panama
> q

----- USSR Player: -----
You may headline any of these cards. Type in the card index.
1	Asia_Scoring
2	Europe_Scoring
3	Middle_East_Scoring
4	Duck_and_Cover - 3
5	Five_Year_Plan - 3
7	Socialist_Governm

AttributeError: 'NoneType' object has no attribute 'info'